In [12]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from schemas import DataCatalog
import constants as const
from ons_population import ONSPopulationScenario

In [2]:
# load data catalog
catalog =  DataCatalog.load_from_yaml("data_catalog.yaml")

In [3]:
OLD_AGE_OVER =65
WORKING_AGE=20

# OADR = number of pension in a population / working age in a population

In [ ]:
# HYPOTHESIS : OADR could be a proxy for population complexity and therefore number of GP appointments per annum

# calculate OADR for each ICB in 2021-22 (FY) and compare to number of appointments (per head of population), is there a correlation?
# fit a model (linear sklearn)

In [106]:
# this is the first in the dataset list
gp_appts_catalog_entry = catalog.single_data_sources[0]
gp_appts_df = gp_appts_catalog_entry.load()
print(gp_appts_catalog_entry.zip_file.namelist())
print(gp_appts_df.shape)

['Regional_CSV_WYorks.csv', 'Regional_CSV_Sussex.csv', 'Regional_CSV_SurreyHearts.csv', 'Regional_CSV_SuffolkNEEssex.csv', 'Regional_CSV_StaffsStoke.csv', 'Regional_CSV_Somerset.csv', 'Regional_CSV_ShropsTelfordWrekin.csv', 'Regional_CSV_SYorks.csv', 'Regional_CSV_SWLondon.csv', 'Regional_CSV_SELondon.csv', 'Regional_CSV_NottinghamNotts.csv', 'Regional_CSV_Northamptonshire.csv', 'Regional_CSV_NorfolkWaveney.csv', 'Regional_CSV_NWLondon.csv', 'Regional_CSV_NENCumbria.csv', 'Regional_CSV_NELondon.csv', 'Regional_CSV_NCLondon.csv', 'Regional_CSV_MidSEssex.csv', 'Regional_CSV_Lincolnshire.csv', 'Regional_CSV_LeicesterLshire.csv', 'Regional_CSV_LancsSCumbria.csv', 'Regional_CSV_KentMedway.csv', 'Regional_CSV_HumberNYorks.csv', 'Regional_CSV_HertfordshireWEssex.csv', 'Regional_CSV_HereWorcs.csv', 'Regional_CSV_HampshireIoW.csv', 'Regional_CSV_Gloucestershire.csv', 'Regional_CSV_GManchesterHSCP.csv', 'Regional_CSV_FrimleyHealth.csv', 'Regional_CSV_Dorset.csv', 'Regional_CSV_Devon.csv', 'Regio

In [108]:
#Combining data for all the ICB's
dfs = []
for ccg_name in gp_appts_catalog_entry.zip_file.namelist():
    gp_appts_catalog_entry.csv_file = ccg_name
    dfs.append(gp_appts_catalog_entry.load())

combined_df = pd.concat(dfs, axis=0)
combined_df.head()

,SUB_ICB_LOCATION_CODE,SUB_ICB_LOCATION_ONS_CODE,SUB_ICB_LOCATION_NAME,ICB_ONS_CODE,ICB_NAME,REGION_ONS_CODE,REGION_NAME,APPOINTMENT_MONTH,APPT_STATUS,HCP_TYPE,APPT_MODE,TIME_BETWEEN_BOOK_AND_APPT,COUNT_OF_APPOINTMENTS
0,02T,E38000025,NHS West Yorkshire ICB - 02T,E54000054,NHS West Yorkshire Integrated Care Board,E40000012,North East and Yorkshire,MAR2021,Attended,GP,Face-to-Face,1 Day,1999
1,02T,E38000025,NHS West Yorkshire ICB - 02T,E54000054,NHS West Yorkshire Integrated Care Board,E40000012,North East and Yorkshire,MAR2021,Attended,GP,Face-to-Face,15 to 21 Days,329
2,02T,E38000025,NHS West Yorkshire ICB - 02T,E54000054,NHS West Yorkshire Integrated Care Board,E40000012,North East and Yorkshire,MAR2021,Attended,GP,Face-to-Face,2 to 7 Days,4194
3,02T,E38000025,NHS West Yorkshire ICB - 02T,E54000054,NHS West Yorkshire Integrated Care Board,E40000012,North East and Yorkshire,MAR2021,Attended,GP,Face-to-Face,22 to 28 Days,212
4,02T,E38000025,NHS West Yorkshire ICB - 02T,E54000054,NHS West Yorkshire Integrated Care Board,E40000012,North East and Yorkshire,MAR2021,Attended,GP,Face-to-Face,8 to 14 Days,1101


In [109]:
#Grouping the data in a series by ICB name and appointments count
appointments_icb = combined_df.groupby('ICB_NAME')['COUNT_OF_APPOINTMENTS'].count()

#Converting series to Dataframe    
appointments_df = pd.DataFrame(appointments_icb)
#sorted_appointments_df = appointments_df.sort_values(by='COUNT_OF_APPOINTMENTS',ascending=False)    
print(appointments_df)

                                                    COUNT_OF_APPOINTMENTS
ICB_NAME                                                                 
NHS Bath and North East Somerset, Swindon and W...                   5588
NHS Bedfordshire, Luton and Milton Keynes Integ...                   5547
NHS Birmingham and Solihull Integrated Care Board                    8191
NHS Black Country Integrated Care Board                              7336
NHS Bristol, North Somerset and South Glouceste...                   7133
NHS Buckinghamshire, Oxfordshire and Berkshire ...                  19191
NHS Cambridgeshire and Peterborough Integrated ...                   7464
NHS Cheshire and Merseyside Integrated Care Board                   47822
NHS Cornwall and the Isles of Scilly Integrated...                   6933
NHS Coventry and Warwickshire Integrated Care B...                   5971
NHS Derby and Derbyshire Integrated Care Board                       6724
NHS Devon Integrated Care Board       

In [29]:
# this is the fifth in the dataset list
gp_appts_catalog_entry4 = catalog.single_data_sources[4]
gp_appts_df2 = gp_appts_catalog_entry4.load()
print(gp_appts_catalog_entry4.zip_file.namelist())
print(gp_appts_df2.shape)
gp_appts_df2.head()

['1. General Practice – August 2023 Individual Level.csv', '0. General Practice Month Year Individual Level Definitions.xlsx', '0. Guide to using Pivot Tables with the Individual-level CSV.pdf']
(200195, 19)


/workspaces/PrimaryCareDemandAndCapacity/schemas.py:144: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(csv_file, **read_csv_kwargs)


,YEAR,Month,COMM_REGION_CODE,COMM_REGION_NAME,ICB_CODE,ICB_NAME,SUB_ICB_CODE,SUB_ICB_NAME,DATA_SOURCE,UNIQUE_IDENTIFIER,STAFF_GROUP,DETAILED_STAFF_ROLE,STAFF_ROLE,COUNTRY_QUALIFICATION_AREA,COUNTRY_QUALIFICATION_GROUP,AGE_BAND,AGE_YEARS,GENDER,FTE
0,2023,8,Y62,North West,QE1,NHS Lancashire and South Cumbria ICB,01K,NHS Lancashire and South Cumbria ICB - 01K Mor...,Provided,33144,GP,Partner/Provider,GP Partners,Asia South,Elsewhere,45-49,49.0,Male,1.013333
1,2023,8,Y63,North East and Yorkshire,QHM,NHS North East and North Cumbria ICB,84H,NHS North East and North Cumbria ICB - 84H Cou...,Provided,134521,Direct Patient Care,Other,Other Direct Patient Care,Not Applicable,Not Applicable,30-34,34.0,Female,0.986667
2,2023,8,Y63,North East and Yorkshire,QHM,NHS North East and North Cumbria ICB,16C,NHS North East and North Cumbria ICB - 16C Tee...,Provided,37777,GP,Senior Partner,GP Partners,EEA,EEA,60-64,60.0,Male,1.000000
3,2023,8,Y63,North East and Yorkshire,QHM,NHS North East and North Cumbria ICB,84H,NHS North East and North Cumbria ICB - 84H Cou...,Provided,83374,GP,Salaried By Practice,Salaried GPs,UK,UK,30-34,31.0,Male,0.560000
4,2023,8,Y63,North East and Yorkshire,QHM,NHS North East and North Cumbria ICB,84H,NHS North East and North Cumbria ICB - 84H Cou...,Provided,183380,Admin/Non-Clinical,Medical Secretary,Medical Secretaries,Not Applicable,Not Applicable,60-64,63.0,Female,0.850133


In [77]:
retired_pop = gp_appts_df2[gp_appts_df2['AGE_YEARS'] >= 65]
working_pop = gp_appts_df2[(gp_appts_df2['AGE_YEARS'] >= 20) & (gp_appts_df2['AGE_YEARS'] < 65)]

#Grouping the data in a series by ICB name and Age
retired_count = retired_pop.groupby('ICB_NAME')['AGE_YEARS'].count()
working_count = working_pop.groupby('ICB_NAME')['AGE_YEARS'].count()

#Calculting the dependency ratio 
dependency_ratio = (retired_count/working_count) * 100

#converting series to Dataframe
oadr_df = pd.DataFrame(dependency_ratio)
#sorted_oadr_df = oadr_df.sort_values(by='AGE_YEARS',ascending=False)
print(oadr_df)

                                                    AGE_YEARS
ICB_NAME                                                     
NHS Bath and North East Somerset, Swindon and W...   4.938989
NHS Bedfordshire, Luton and Milton Keynes ICB        6.552806
NHS Birmingham and Solihull ICB                      6.026447
NHS Black Country ICB                                6.370450
NHS Bristol, North Somerset and South Glouceste...   4.909140
NHS Buckinghamshire, Oxfordshire and Berkshire ...   8.192040
NHS Cambridgeshire and Peterborough ICB              6.731392
NHS Cheshire and Merseyside ICB                      6.003329
NHS Cornwall and The Isles Of Scilly ICB             5.050505
NHS Coventry and Warwickshire ICB                    7.413914
NHS Derby and Derbyshire ICB                         4.526861
NHS Devon ICB                                        5.158981
NHS Dorset ICB                                       6.413302
NHS Frimley ICB                                      9.786223
NHS Glou

# **Combining both the series - Appointments and OADR based on ICB**

In [105]:
#Changing 'Integrated Care Board' to 'ICB' in the index name to match the index of both the dataframes
for old_word, new_word in {'Integrated Care Board': 'ICB'}.items():
    appointments_df.index = appointments_df.index.str.replace(old_word, new_word )
    
#Combining both dataframes
appointments_vs_oadr = pd.concat([appointments_df,oadr_df], axis=1)

#sorting
sorted_appointments_vs_oadr = appointments_vs_oadr.sort_values(by='AGE_YEARS',ascending=False)
sorted_appointments_vs_oadr.head(50)

,COUNT_OF_APPOINTMENTS,AGE_YEARS
ICB_NAME,,
NHS North West London ICB,7138.0,10.457866
NHS South West London ICB,7559.0,10.314016
NHS Mid and South Essex ICB,23697.0,9.807976
NHS Frimley ICB,5763.0,9.786223
NHS Surrey Heartlands ICB,6729.0,9.186697
NHS North East London ICB,7302.0,9.036952
NHS North Central London ICB,6387.0,8.846052
NHS Hertfordshire and West Essex ICB,16795.0,8.809135
NHS South East London ICB,7364.0,8.517961


In [38]:
ons_principal_projection = ONSPopulationScenario(catalog.scenario_data_sources[0].scenarios[0])
ons_df = ons_principal_projection.load()

AttributeError: 'ONSPopulationScenario' object has no attribute 'load'